In [ ]:
import collections
import hashlib
import random
import binascii
import sys
from Crypto.Cipher import AES
import Padding

In [6]:
def enc_long(n):
    s = ""
    while n > 0:
        s = chr(n & 0xFF) + s
        n >>= 8
    return s

In [7]:
BLOCK_SIZE = 16
pad = lambda s: s + (BLOCK_SIZE - len(s) % BLOCK_SIZE) * \
                chr(BLOCK_SIZE - len(s) % BLOCK_SIZE)
unpad = lambda s: s[:-ord(s[len(s) - 1:])]

In [8]:
def encrypt(plaintext,key, mode):
    encobj = AES.new(key,mode)
    return(encobj.encrypt(plaintext))


In [9]:
def decrypt(ciphertext,key, mode):
    encobj = AES.new(key,mode)
    return(encobj.decrypt(ciphertext))

In [10]:
EllipticCurve = collections.namedtuple('EllipticCurve', 'name p a b g n h')

In [11]:
curve = EllipticCurve(
    'secp256k1',
    # Field characteristic.
    p=0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f,
    # Curve coefficients.
    a=0,
    b=7,
    # Base point.
    g=(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
       0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8),
    # Subgroup order.
    n=0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141,
    # Subgroup cofactor.
    h=1,
)

In [12]:
def is_on_curve(point):
    if point is None:
        return True
    x, y = point
    return (y * y - x * x * x - curve.a * x - curve.b) % curve.p == 0

In [13]:
def point_neg(point):
    assert is_on_curve(point)
    if point is None:
        return None
    x, y = point
    result = (x, -y % curve.p)
    assert is_on_curve(result)
    return result

In [14]:
def point_add(point1, point2):
    assert is_on_curve(point1)
    assert is_on_curve(point2)

    if point1 is None:
        return point2
    if point2 is None:
        return point1

    x1, y1 = point1
    x2, y2 = point2

    if x1 == x2 and y1 != y2:
        return None

    if x1 == x2:
        m = (3 * x1 * x1 + curve.a) * inverse_mod(2 * y1, curve.p)
    else:
        m = (y1 - y2) * inverse_mod(x1 - x2, curve.p)

    x3 = m * m - x1 - x2
    y3 = y1 + m * (x3 - x1)
    result = (x3 % curve.p,-y3 % curve.p)

    assert is_on_curve(result)
    return result

In [15]:
def scalar_mult(k, point):
    assert is_on_curve(point)

    if k % curve.n == 0 or point is None:
        return None

    if k < 0:
        return scalar_mult(-k, point_neg(point))

    result = None
    addend = point

    while k:
        if k & 1:
            result = point_add(result, addend)
        addend = point_add(addend, addend)
        k >>= 1

    assert is_on_curve(result)
    return result

In [16]:
def make_keypair():
    private_key = random.randrange(1, curve.n)
    public_key = scalar_mult(private_key, curve.g)
    return private_key, public_key

In [17]:
def verify_signature(public_key, message, signature):
    m = hash_message(message)

    r, s = signature

    w = inverse_mod(s, curve.n)
    u1 = (z * w) % curve.n
    u2 = (r * w) % curve.n

    x, y = point_add(scalar_mult(u1, curve.g),
                     scalar_mult(u2, public_key))

    if (r % curve.n) == (x % curve.n):
        return 'signature matches'
    else:
        return 'invalid signature'

In [19]:
message="AnhQuan"
if (len(sys.argv)>1):
	message=str(sys.argv[1])

In [20]:
dA, Qa = make_keypair()
print("Private key:", hex(dA))
print(("Public key: (0x{:x}, 0x{:x})".format(*Qa)))

Private key: 0xa58c58dc27e60918c20cb026d2edb16b5e8462b6864f6eea64660570b01995e1
Public key: (0x6a536b68362ed3adf8d92bce81ace86a485caf4c5b93d8aec84c5d19f30c78c6, 0x18caead55c444a405d3f22dfe196a646f257831cc4b297c57a3ef62a5dcb9cdb)


In [21]:
r = random.randint(0, 2**128)

rG = scalar_mult(r,curve.g)
S = scalar_mult(r,Qa)
print("Random value: " , r)
print("rG: " , rG)
print("\n\n======Symmetric key========")
print("Encryption key:",S[0],str(S[0]))
password='hello'

Random value:  198190841552508904013102987000789748734
rG:  (15188870034828594851470019924690283117300374100814987147435792823330674996999, 15094067035241856690472202899189594022004495979290067480618050230806439003261)


======Symmetric key========
Encryption key: 77147593802058067386001286845947471919716179267284048446532052395920572379612 77147593802058067386001286845947471919716179267284048446532052395920572379612


In [ ]:
key = hashlib.sha256(str(S[0]).encode()).digest()

message = Padding.appendPadding(message,blocksize=Padding.AES_blocksize,mode=0)

ciphertext = encrypt(message.encode(),key,AES.MODE_ECB)
print("Encrypted:\t",binascii.hexlify(ciphertext))

In [ ]:
Snew = scalar_mult(dA,rG)
key = hashlib.sha256(str(Snew[0]).encode()).digest()

text = decrypt(ciphertext,key,AES.MODE_ECB)

print("Decrypted:\t",Padding.removePadding(text.decode(),mode=0))